In [ ]:
!pip install google-cloud-secret-manager

# 1. Create BQ table from scratch

In [ ]:
from google.cloud import bigquery
from google.cloud.bigquery import SchemaField
from google.cloud import secretmanager
import google.auth.transport.requests

secret_client = secretmanager.SecretManagerServiceClient()
_, project_number = google.auth.default()

def get_secret(secret_name):
    secret_path = f"projects/{project_number}/secrets/{secret_name}/versions/latest"
    response = secret_client.access_secret_version(name=secret_path)
    return response.payload.data.decode("UTF-8")

# Table details
table_id = get_secret("BIGQUERY_TABLE_LIST")  # Replace with your BigQuery table name

# Initialize BigQuery client
bq_client = bigquery.Client()

# Define the schema for the table
schema = [
    bigquery.SchemaField("file_number", "INTEGER", mode="NULLABLE"),         # Sequential number of the file
    bigquery.SchemaField("added_to_list", "DATETIME", mode="NULLABLE"),     # When the file was added to the list
    bigquery.SchemaField("file_id", "STRING", mode="REQUIRED"),          # Unique identifier for the file
    bigquery.SchemaField("file_name", "STRING", mode="NULLABLE"),       # Name of the file
    bigquery.SchemaField("file_created_time", "TIMESTAMP", mode="NULLABLE"), # Creation time of the file
    bigquery.SchemaField("parent_folder_id", "STRING", mode="NULLABLE"),    # ID of the parent folder
    bigquery.SchemaField("web_view_link", "STRING", mode="NULLABLE"),   # Web view link of the file
    bigquery.SchemaField("file_path", "STRING", mode="NULLABLE"),       # Full path of the file
    bigquery.SchemaField("file_mime_type", "STRING", mode="NULLABLE"),  # MIME type of the file
    bigquery.SchemaField("file_size", "FLOAT", mode="NULLABLE"),        # Size of the file in MB
    bigquery.SchemaField("copied_to_gcs", "BOOL", mode="NULLABLE"),     # Whether the file is copied to GCS
    bigquery.SchemaField("timestamp_copied", "DATETIME", mode="NULLABLE"), # Timestamp when the file was copied
    bigquery.SchemaField("parsed", "STRING", mode="NULLABLE"),            # Whether the file is parsed
    bigquery.SchemaField("timestamp_parsed", "DATETIME", mode="NULLABLE") # Timestamp when the file was parsed
]


# Define the table
table = bigquery.Table(table_id, schema=schema)

# Create the table
try:
    created_table = bq_client.create_table(table)
    print(f"Table {table_id} created successfully.")
except Exception as e:
    print(f"Error creating table: {e}")

# 2. Add new columns to the existing table

In [ ]:
from google.cloud import bigquery
from google.cloud.bigquery import SchemaField
from google.cloud import secretmanager
import google.auth.transport.requests

secret_client = secretmanager.SecretManagerServiceClient()
_, project_number = google.auth.default()

def get_secret(secret_name):
    secret_path = f"projects/{project_number}/secrets/{secret_name}/versions/latest"
    response = secret_client.access_secret_version(name=secret_path)
    return response.payload.data.decode("UTF-8")

# Table details
table_id = get_secret("BIGQUERY_TABLE_LIST")  # existing table name

# Initialize BigQuery client
bq_client = bigquery.Client()

# Define the new columns you want to add
new_columns = [
    bigquery.SchemaField("added_to_bq", "BOOL", mode="NULLABLE"),
    bigquery.SchemaField("timestamp_added", "DATETIME", mode="NULLABLE")
]

try:
    # Get the current table
    table = bq_client.get_table(table_id)

    # Convert the existing schema to a list so we can modify it
    current_schema = list(table.schema)

    # Get the names of existing columns (to avoid duplicates)
    existing_column_names = {field.name for field in current_schema}

    # Only add new columns that do not already exist in the table
    columns_to_add = [
        col for col in new_columns
        if col.name not in existing_column_names
    ]

    if columns_to_add:
        # Update the schema by appending the new columns
        updated_schema = current_schema + columns_to_add

        # Assign the updated schema back to the table
        table.schema = updated_schema

        # Update the table in BigQuery
        table = bq_client.update_table(table, ["schema"])

        print(f"Successfully added columns {[col.name for col in columns_to_add]} to {table_id}")
    else:
        print("No new columns to add. All columns already exist.")

except Exception as e:
    print(f"Error modifying table schema: {e}")
